In [61]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
import os
import pickle

In [62]:
def extract_color_histogram(frame, bins=32):
    """ Trích xuất histogram màu sắc từ khung hình dưới dạng HSV. """
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, [bins]*3, [0, 180, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

In [63]:
def extract_lbp_features(frame, radii=[1, 3, 5], n_points=36, method='uniform'):
    """ Trích xuất đặc trưng Local Binary Patterns từ khung hình với nhiều bán kính. """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    lbp_features = []
    for radius in radii:
        lbp = local_binary_pattern(gray, n_points, radius, method)
        hist, _ = np.histogram(lbp.ravel(), bins=n_points+2, range=(0, n_points+2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-6)
        lbp_features.extend(hist)
    return np.array(lbp_features)

In [64]:
def extract_sift_features(frame):
    """
    Trích xuất đặc trưng SIFT từ khung hình.
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return descriptors

In [65]:
def process_video(video_path, sample_rate=30, bins=32, radii=[1, 3, 5], n_points=36, sift_dim=128):
    cap = cv2.VideoCapture(video_path)
    color_features = []
    texture_features = []
    sift_features = []
    frame_index = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_index % sample_rate == 0:
            color_hist = extract_color_histogram(frame, bins=bins)
            lbp_hist = extract_lbp_features(frame, radii=radii, n_points=n_points)
            sift_desc = extract_sift_features(frame)
            if sift_desc is not None:
                if sift_desc.shape[1] != sift_dim:
                    padded_desc = np.zeros((sift_desc.shape[0], sift_dim), dtype=np.float32)
                    padded_desc[:, :sift_desc.shape[1]] = sift_desc
                    sift_desc = padded_desc
                sift_desc = sift_desc.reshape(sift_desc.shape[0], 1, sift_dim)
            color_features.append(color_hist)
            texture_features.append(lbp_hist)
            sift_features.append(sift_desc)
        frame_index += 1

    cap.release()
    # Đảm bảo làm phẳng và trung bình mọi thứ một cách thích hợp
    if color_features:
        color_features = np.mean(np.array(color_features), axis=0)
    if texture_features:
        texture_features = np.mean(np.array(texture_features), axis=0)
    if sift_features:
        sift_features = [feat for feat in sift_features if feat is not None]
        if sift_features:
            min_keypoints = min(feat.shape[0] for feat in sift_features)
            sift_features = [feat[:min_keypoints] for feat in sift_features]
            sift_features = np.concatenate(sift_features, axis=1)
            sift_features = np.mean(sift_features, axis=0)
        else:
            sift_features = None

    return color_features, texture_features, sift_features

In [66]:
def save_features(features, file_name):
    """ Lưu đặc trưng ra file. """
    with open(file_name, 'wb') as f:
        pickle.dump(features, f)

In [67]:
def process_all_videos(directory_path, sample_rate=30):
    """Process all video files in the directory and save the .pkl files in a 'data' subdirectory."""
    # Create a directory for the .pkl files if it doesn't exist
    data_directory = os.path.join('/kaggle/working/', 'data')
    os.makedirs(data_directory, exist_ok=True)

    features_database = {}

    for file_name in os.listdir(directory_path):
        if file_name.endswith('.mp4'):  # Check if the file is a video
            video_path = os.path.join(directory_path, file_name)
            color_features, texture_features, sift_features = process_video(video_path, sample_rate)

            # Save the features in the 'data' subdirectory
            save_features(color_features, os.path.join(data_directory, f'{file_name}_color_features.pkl'))
            save_features(texture_features, os.path.join(data_directory, f'{file_name}_texture_features.pkl'))
            save_features(sift_features, os.path.join(data_directory, f'{file_name}_sift_features.pkl'))

            # Lưu đặc trưng của video vào features_database
            features_database[file_name] = (color_features, texture_features, sift_features)

            print(f"Đã trích xuất và lưu đặc trưng cho video {file_name}")

    return features_database

In [68]:
def calculate_distance(color_features, texture_features, sift_features, other_color, other_texture, other_sift):
    color_dist = np.linalg.norm(color_features - other_color)
    texture_dist = np.linalg.norm(texture_features - other_texture)
    sift_dist = np.linalg.norm(sift_features - other_sift)

    # Áp dụng trọng số cho các đặc trưng
    weighted_dist = 0.3 * color_dist + 0.2 * texture_dist + 0.5 * sift_dist

    return weighted_dist

In [69]:
def find_similar_videos(input_video_path, features_database, sample_rate=30):
    input_color_features, input_texture_features, input_sift_features = process_video(input_video_path, sample_rate)

    distances = []
    for video_name, (color_features, texture_features, sift_features) in features_database.items():
        distance = calculate_distance(input_color_features, input_texture_features, input_sift_features,
                                      color_features, texture_features, sift_features)
        distances.append((video_name, distance))

    distances.sort(key=lambda x: x[1])  # Sắp xếp theo khoảng cách tăng dần

    return [video_name for video_name, _ in distances[:5]]  # Trả về 5 video có khoảng cách nhỏ nhất


In [70]:
directory_path = '/kaggle/input/gogogomeomeomeo4/cho-meo'
features_database = process_all_videos(directory_path, sample_rate=30)


Đã trích xuất và lưu đặc trưng cho video 123.mp4
Đã trích xuất và lưu đặc trưng cho video 033.mp4
Đã trích xuất và lưu đặc trưng cho video 113.mp4
Đã trích xuất và lưu đặc trưng cho video 106.mp4
Đã trích xuất và lưu đặc trưng cho video 019.mp4
Đã trích xuất và lưu đặc trưng cho video 017.mp4
Đã trích xuất và lưu đặc trưng cho video 081.mp4
Đã trích xuất và lưu đặc trưng cho video 114.mp4
Đã trích xuất và lưu đặc trưng cho video 042.mp4
Đã trích xuất và lưu đặc trưng cho video 058.mp4
Đã trích xuất và lưu đặc trưng cho video 120.mp4
Đã trích xuất và lưu đặc trưng cho video 026.mp4
Đã trích xuất và lưu đặc trưng cho video 078.mp4
Đã trích xuất và lưu đặc trưng cho video 085.mp4
Đã trích xuất và lưu đặc trưng cho video 030.mp4
Đã trích xuất và lưu đặc trưng cho video 045.mp4
Đã trích xuất và lưu đặc trưng cho video 088.mp4
Đã trích xuất và lưu đặc trưng cho video 093.mp4
Đã trích xuất và lưu đặc trưng cho video 055.mp4
Đã trích xuất và lưu đặc trưng cho video 009.mp4
Đã trích xuất và lưu